In [5]:
import numpy as np
import librosa
import scipy


In [2]:
def extract_pitch(y, sr, fmin=50, fmax=300):
    """
    Extract the fundamental frequency (F0) using librosa's autocorrelation method.

    Parameters:
    - y: np.ndarray, audio time series.
    - sr: int, sampling rate.
    - fmin: float, minimum frequency to consider.
    - fmax: float, maximum frequency to consider.

    Returns:
    - pitches: np.ndarray, pitch (F0) values over time.
    - magnitudes: np.ndarray, magnitudes of the detected pitches.
    """
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, fmin=fmin, fmax=fmax)
    
    # Extract the pitch for each frame
    pitch_values = []
    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()
        pitch = pitches[index, t]
        pitch_values.append(pitch if pitch > 0 else np.nan)  # Use NaN for no pitch detected
    
    return np.array(pitch_values)

# Usage
file_path = r'C:\Users\syrym\Downloads\research_2\audio.wav'
y, sr = librosa.load(file_path, sr=None)
pitches = extract_pitch(y, sr)


In [3]:
def classify_gender(pitches):
    """
    Classify the gender based on the average pitch.

    Parameters:
    - pitches: np.ndarray, pitch (F0) values over time.

    Returns:
    - str, predicted gender ('Male' or 'Female').
    """
    avg_pitch = np.nanmean(pitches)
    print(f"Average Pitch: {avg_pitch:.2f} Hz")
    
    if avg_pitch > 165:
        return 'Female'
    else:
        return 'Male'

# Usage
predicted_gender = classify_gender(pitches)
print(f"Predicted Gender: {predicted_gender}")


Average Pitch: 165.22 Hz
Predicted Gender: Female


In [7]:
from scipy.signal import lfilter

def extract_formants(y, sr, frame_length=0.025, hop_length=0.01):
    """
    Extract the first two formant frequencies (F1, F2) using LPC.

    Parameters:
    - y: np.ndarray, audio time series.
    - sr: int, sampling rate.
    - frame_length: float, length of the frame (in seconds).
    - hop_length: float, hop length between frames (in seconds).

    Returns:
    - formants: list of tuples, containing F1 and F2 for each frame.
    """
    n_samples_per_frame = int(frame_length * sr)
    hop_length_samples = int(hop_length * sr)
    formants = []

    for i in range(0, len(y) - n_samples_per_frame, hop_length_samples):
        frame = y[i:i + n_samples_per_frame]
        
        # Apply LPC
        a = librosa.lpc(frame, 2 + sr // 1000)
        roots = np.roots(a)
        roots = [r for r in roots if np.imag(r) >= 0]  # Keep only positive frequencies
        formant_frequencies = np.angle(roots) * (sr / (2 * np.pi))

        if len(formant_frequencies) >= 2:
            formants.append((formant_frequencies[0], formant_frequencies[1]))
    
    return formants

# Usage
formants = extract_formants(y, sr)


TypeError: lpc() takes 1 positional argument but 2 were given

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def extract_mfccs(y, sr, n_mfcc=13):
    """
    Extract MFCCs from the audio signal.

    Parameters:
    - y: np.ndarray, audio time series.
    - sr: int, sampling rate.
    - n_mfcc: int, number of MFCC coefficients to extract.

    Returns:
    - mfccs: np.ndarray, MFCCs of the audio.
    """
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Split dataset into features (X) and labels (y)
def train_gender_classifier(X, y):
    """
    Train a logistic regression model for gender classification.

    Parameters:
    - X: np.ndarray, feature matrix (MFCCs or other features).
    - y: np.ndarray, labels (gender: 0 for male, 1 for female).

    Returns:
    - model: trained logistic regression model.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    return model

# Example for MFCCs and training
mfccs = extract_mfccs(y, sr)
